In [240]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# PostgreSQL connection parameters
db_username = '<USERNAME>'
db_password = '<PASSWORD>'
db_host = '<HOST>'
db_port = '<PORT>'
db_name = '<DATABASE NAME>'
# Construct the connection string
connection_string = f"dbname={db_name} user={db_username} password={db_password} host={db_host} port={db_port}"

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Directory where your HTML files are stored
directory = r'\pages'

### Funtions

In [241]:
##############################################################################################################
# This function will parse the concatenated attribute from Attack stats and split it into three columns      #
##############################################################################################################

def split_and_convert_column(df, column_name):
    pattern = r'(\d+)/(\d+) \((\d+)%\)'

    # Replace 0 with "0/0 (0%)"
    df[column_name] = df[column_name].apply(lambda x: "0/0 (0%)" if x == "0" else x)
    
    # Splitting the column into three separate columns
    df[[f'{column_name} Success', f'{column_name} Total', f'{column_name} Percentage']] = df[column_name].str.extract(pattern)

    # Convert columns to numeric values if needed
    df[f'{column_name} Success'] = pd.to_numeric(df[f'{column_name} Success'])
    df[f'{column_name} Total'] = pd.to_numeric(df[f'{column_name} Total'])
    df[f'{column_name} Percentage'] = pd.to_numeric(df[f'{column_name} Percentage'].str.rstrip('%')) / 100

    return df

In [ ]:
##############################################################################################################
# This function will clean the data                                                                          #
##############################################################################################################

def cleaning_data(df):
    # Remove goalkeepers
    df = df[df['Id'] != -1]

    # Convert gameDate column to datetime
    df['gameDate'] = pd.to_datetime(df['gameDate'])

    # Check if exists NaN values in the DataFrame
    nan_columns = df.isna().any()

    # Display columns with NaN values
    columns_with_nan = nan_columns[nan_columns].index.tolist()
    print("Columns with NaN values:", columns_with_nan)

    # Fill NaN or None values with 0 for the identified columns
    df[columns_with_nan] = df[columns_with_nan].fillna(0)

    return df

In [ ]:
##############################################################################################################
# This function will execute the Random Forest Classifier and return the results in a DataFrame              #
##############################################################################################################

from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def random_forest_evaluation(df_random, X, y):

    # Encode categorical variables if needed
    X = pd.get_dummies(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize RandomForestClassifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model
    rf_classifier.fit(X_train, y_train)

    # Predict on the test set
    predictions = rf_classifier.predict(X_test)

    # Evaluate model performance
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: " + str(accuracy))
    classification_rep = classification_report(y_test, predictions)
    # print(classification_rep)

    # Create a dictionary to store unique player predictions
    unique_predictions = {}

    # Populate the dictionary with unique player predictions
    for i, player_idx in enumerate(y_test.index):
        player_name = df_random.loc[player_idx, 'Player Name']  # Assuming 'Player' is the column name containing player names
        unique_predictions[player_name] = predictions[i]

    # Create a DataFrame to display unique player predictions
    result_df = pd.DataFrame(list(unique_predictions.items()), columns=['Player Name', 'Predicted'])
    # Sort the DataFrame by the 'Predicted' column in descending order
    sorted_results_desc = result_df.sort_values(by='Predicted', ascending=True)

    return sorted_results_desc

In [242]:
##############################################################################################################
# This function will extract the players and their stats from the HTML content, save to the database and     #
# return the results in a DataFrame                                                                          #
##############################################################################################################

goalkeepers = []
header_info = list()

# Function to extract players and their stats from HTML content
def extract_players_stats(html_content, gameDate):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract header information
    header_columns = soup.find('thead').find('tr', class_=lambda x: x and 'HeaderRow' in x)
    
    header_info = [th.text for th in header_columns.find_all('th', class_=lambda x: x and 'HeaderCell' in x)]
    header_info.append('Id')
    header_info.append('Player Name')
    header_info.append('Position')
    header_info.append('Team')
    header_info.append('gameDate')

    # Extract information for all players
    all_players_info = []

    df_players = pd.read_csv(r'csv_files/Players.csv')
    # Find all <tr> elements containing player information
    player_rows = soup.find_all('tr', class_=lambda x: x and 'TableRowStyled' in x)

    for row in player_rows:
        player_info = {}
        stats = [span.text for span in row.find_all('td', class_=lambda x: x and 'TableCell' in x)]

        result = df_players[(df_players['Name'] == stats[0]) | (df_players['Fotmob Name'] == stats[0]) | (df_players['Encode Name'] == stats[0])]
        if not result.empty and 'Id' in result.columns:
            stats.append(result['Id'].iloc[0])
            stats.append(result['Fotmob Name'].iloc[0])
            stats.append(result['Position'].iloc[0])
            stats.append(result['Team'].iloc[0])
        else:
            if stats[0] not in goalkeepers:
                goalkeepers.append(stats[0])
            stats.append(-1)
            stats.append(stats[0])
            stats.append('Goalkeeper')
            stats.append('N/A')
        
        # append gameDate
        stats.append(gameDate)

        player_info['Stats'] = stats

        # Append player's information to the list
        all_players_info.append(player_info)

    # Create DataFrame
    df = pd.DataFrame([entry['Stats'] for entry in all_players_info if 'Stats' in entry], columns=header_info)
    # df.to_csv(r'csv_files/' + fileNameCsv, index=False)

    df = split_and_convert_column(df, 'Shots on target')
    df = split_and_convert_column(df, 'Successful dribbles')
    
    # Save the DataFrame to the PostgreSQL database
    df.to_sql('attack', engine, if_exists='append', index=False, method='multi', chunksize=500)
    
    return all_players_info

['N. Ingham', 'Y. Fillion', 'Marco Carducci', 'Triston Henry', 'E. Gazdov', 'Callum Irving', 'R. Yesli', 'Niko Giantsopoulos', 'Sean Melvin', 'C. Kalongo', 'K. Baskett', 'E. Himaras', 'J. Zielinski', 'Darlington Murasiranwa', 'Yann-Alexandre Fillion', 'Nathan Ingham']
3533


### Execution

In [243]:
# Open the folder containing HTML files, loop through each file and extract players and their stats and save to the database

dataset = list()
# Loop through each HTML file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.html'):
        file_path = os.path.join(directory, filename)
        
        # Read HTML content from file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # print(f"File: {filename}")

        fileNameCsv = filename.replace('.html', '.csv')
        # Extract players and their stats from HTML content
        players_stats = extract_players_stats(html_content, fileNameCsv[:10])

        dataset.extend(players_stats)

if len(goalkeepers) > 0:
    print(goalkeepers)

print(len(dataset))

### Queries to verify the consistency of data

In [ ]:
'''

-- Query to verify if the Player Name had more than one name. If so, I inserted an unique name in the "Player Name" column
select * from attack where attack."Player" like '%Rosario%'

-- Query to verify the monthly rank
select distinct(attack."Player Name"), attack."Team", attack."Position", attack."Ranking", attack."Final Ranking"
from attack 
where EXTRACT(MONTH FROM attack."gameDate"::date) = 5
order by attack."Ranking";

-- Query to verify the season rank
select distinct(attack."Player Name"), attack."Team", attack."Position", attack."Final Ranking"
from attack 
order by attack."Final Ranking";


-- Checking if there are any null values in the Ranking column
SELECT count(*) FROM attack where attack."Ranking" is not null

-- Checking the players have more than five goals on the season
select * from attack where attack."Id" in(
						SELECT 
                        attack."Id"
                        FROM attack 
                        WHERE attack."Position" = 'Forward' or attack."Position" = 'Midfielder'
                        group by  
                        attack."Id"
                        having sum(CAST(attack."Goals" as numeric)) >= 5)

                        
-- Checking if there is no name duplicated in the Player Name column
SELECT attack."Id"
FROM attack
GROUP BY attack."Id"
HAVING COUNT(DISTINCT attack."Player Name") > 1;


'''

### Cleaning before running Random Forest Classifier

In [244]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(connection_string)

# Execute the query and fetch the results into a Pandas DataFrame
df1 = pd.read_sql_query('''
                        SELECT 
                        attack."Id", 
                        attack."gameDate", 
                        attack."Player Name", 
                        attack."Goals", 
                        attack."Total shots", 
                        attack."Shots on target Success", 
                        attack."Shots on target Total", 
                        attack."Shots on target Percentage",
                        attack."Successful dribbles Success", 
                        attack."Successful dribbles Total", 
                        attack."Successful dribbles Percentage",
                        attack."Big chances missed", 
                        attack."Offsides", 
                        attack."Dispossessed", 
                        attack."Touches",
                        attack."Ranking"
                        FROM attack 
                        WHERE attack."Position" = 'Forward' or attack."Position" = 'Midfielder'
                        group by  
                        attack."Id", 
                        attack."gameDate", 
                        attack."Player Name", 
                        attack."Goals", 
                        attack."Total shots", 
                        attack."Shots on target Success", 
                        attack."Shots on target Total", 
                        attack."Shots on target Percentage",
                        attack."Successful dribbles Success", 
                        attack."Successful dribbles Total", 
                        attack."Successful dribbles Percentage",
                        attack."Big chances missed", 
                        attack."Offsides", 
                        attack."Dispossessed", 
                        attack."Touches",
                        attack."Ranking"
                        '''
                        , conn)

# Close the database connection
conn.close()

df1 = cleaning_data(df1)

Columns with NaN values: ['Big chances missed', 'Offsides', 'Touches', 'Ranking']


C:\Users\leoau\AppData\Local\Temp\ipykernel_9548\1759354406.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query('''


### Establishing the monthly rank

In [246]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()

for desired_month in range(4, 11):
    print(desired_month)
    filtered_df = df1[df1['gameDate'].dt.month == desired_month]

    # Define features (X) and target (y)
    X = pd.DataFrame(filtered_df, columns=['Player Name', 'Goals', 'Total shots', 
                'Shots on target Success', 'Shots on target Total', 'Shots on target Percentage',
                'Successful dribbles Success', 'Successful dribbles Total', 'Successful dribbles Percentage',
                'Big chances missed', 'Offsides', 'Dispossessed', 'Touches'])
    y = pd.DataFrame(filtered_df, columns=['Ranking'])  # Replace 'BestPlayerColumn' with your target column

    results_df = random_forest_evaluation(filtered_df, X, y)

    results_df['Ranking'] = range(1, len(results_df) + 1)


    # Iterate through rows and update the database
    for index, row in results_df.iterrows():
        record_player = row['Player Name']
        new_value = row['Ranking']

        # Construct and execute the SQL update statement
        update_query = f'UPDATE attack SET "Ranking" = ' + str(new_value) + ' WHERE "Player Name" = ' + "'" + record_player + "'" + ' AND EXTRACT(MONTH FROM "gameDate"::date) = ' + str(desired_month)
        cursor.execute(update_query)

# Commit the changes and close the database connection
conn.commit()
conn.close()

4


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0
5


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0
6


...

Accuracy: 1.0
9


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0
10


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0


### Establishing the season rank

In [247]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(connection_string)

# Execute the query and fetch the results into a Pandas DataFrame
df1 = pd.read_sql_query('''
                        SELECT * FROM attack 
                        WHERE attack."Id" in(
                            SELECT 
                            attack."Id"
                            FROM attack 
                            WHERE attack."Position" = 'Forward' or attack."Position" = 'Midfielder'
                            group by  
                            attack."Id"
                            having sum(CAST(attack."Goals" as numeric)) >= 5
                        )
                        '''
                        , conn)
df1 = cleaning_data(df1)


# Define features (X) and target (y)
X = pd.DataFrame(df1, columns=['Player Name', 'Goals', 'Total shots', 
            'Shots on target Success', 'Shots on target Total', 'Shots on target Percentage',
            'Successful dribbles Success', 'Successful dribbles Total', 'Successful dribbles Percentage',
            'Big chances missed', 'Offsides', 'Dispossessed', 'Touches'])
y = pd.DataFrame(df1, columns=['Final Ranking'])  # Replace 'BestPlayerColumn' with your target column

results_df = random_forest_evaluation(df1, X, y)

results_df['Final Ranking'] = range(1, len(results_df) + 1)

# Iterate through rows and update the database
cursor = conn.cursor()
for index, row in results_df.iterrows():
    record_player = row['Player Name']
    new_value = row['Final Ranking']

    # Construct and execute the SQL update statement
    update_query = f'UPDATE attack SET "Final Ranking" = ' + str(new_value) + ' WHERE "Player Name" = ' + "'" + record_player + "'" + ' AND EXTRACT(MONTH FROM "gameDate"::date) = ' + str(desired_month)
    cursor.execute(update_query)

# Commit the changes and close the database connection
conn.commit()
conn.close()

C:\Users\leoau\AppData\Local\Temp\ipykernel_9548\2083369144.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query('''
C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Columns with NaN values: ['Big chances missed', 'Touches in opposition box', 'Offsides', 'Touches', 'Ranking', 'Final Ranking']
Accuracy: 1.0
Accuracy: 1.0


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0
Accuracy: 1.0


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0
Accuracy: 1.0


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 1.0


C:\Users\leoau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


### Creating the dataset

In [248]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(connection_string)

# Execute the query and fetch the results into a Pandas DataFrame
df1 = pd.read_sql_query("SELECT * FROM attack", conn)

# Close the database connection
conn.close()

df1 = cleaning_data(df1)

df1.to_csv(r'csv_files/dataset.csv', index=False)

Columns with NaN values: ['Big chances missed', 'Touches in opposition box', 'Offsides', 'Touches', 'Ranking', 'Final Ranking']


C:\Users\leoau\AppData\Local\Temp\ipykernel_9548\2518040848.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query("SELECT * FROM attack", conn)
